In [10]:
# Menghitung dua pangkat dari 1 hingga 20
for i in range(1, 21):
    print(f"2^{i} = {2**i}")

2^1 = 2
2^2 = 4
2^3 = 8
2^4 = 16
2^5 = 32
2^6 = 64
2^7 = 128
2^8 = 256
2^9 = 512
2^10 = 1024
2^11 = 2048
2^12 = 4096
2^13 = 8192
2^14 = 16384
2^15 = 32768
2^16 = 65536
2^17 = 131072
2^18 = 262144
2^19 = 524288
2^20 = 1048576


In [11]:
import math

int(math.log2(3600))

11

In [12]:
print(bin(460)[2:])
print(bin(2)[2:])
print(bin(4)[2:].zfill(11))


111001100
10
00000000100


In [13]:
from collections import OrderedDict
import pickle

from ml_pee_stego_v2 import MLPEEStego

# Set up the secret

In [14]:
from utils.data_preparation import get_filenames_from_folder, get_secret_file

# Tentukan folder path tempat secret
SECRET_FOLDER_PATH = 'secrets/bin'

# Dapatkan semua nama file .txt dari folder
secret_filenames = get_filenames_from_folder('txt', SECRET_FOLDER_PATH)
secret_filenames.sort(reverse=True)
print(secret_filenames)

# Masukkan ke dalam dictionary berisi nama filename beserta isi secret filenya
secret_dicts = OrderedDict((secret_filename, get_secret_file(
    SECRET_FOLDER_PATH + '/' + secret_filename + '.txt')) for secret_filename in secret_filenames)
# print(secret_dicts.get(secret_filenames[1]))

for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")

['secret_0.39_bps', 'secret_0.32_bps', 'secret_0.21_bps', 'secret_0.15_bps', 'secret_0.08_bps']
Secret untuk secret_0.39_bps
Secret untuk secret_0.32_bps
Secret untuk secret_0.21_bps
Secret untuk secret_0.15_bps
Secret untuk secret_0.08_bps


# Set up the model

In [15]:
import warnings
from sklearn.exceptions import InconsistentVersionWarning

# Menonaktifkan peringatan InconsistentVersionWarning
warnings.simplefilter("ignore", category=InconsistentVersionWarning)

models = OrderedDict()

models['SVR'] = 'models/svr_model.pkl'
models['SVR 2.0'] = 'models/svr_model_2.0.pkl'
models['LASSO'] = 'models/lasso_model.pkl'
models['ElasticNet'] = 'models/elastic_net_model.pkl'
models['Bayesian Ridge'] = 'models/bayesian_ridge_model.pkl'
models['SGD Regressor'] = 'models/sgd_model.pkl'

In [16]:
from utils.data_preparation import get_original_data

patient_code = '100'
original_data = get_original_data(patient_code=patient_code)

In [17]:
# Example
model_name = 'LASSO'
with open(models.get(model_name), 'rb') as model_file:
    model = pickle.load(model_file)

stego = MLPEEStego(model)
threshold = 4
for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")
    for key in ['001', '010', '011', '100', '101', '110']:
        print(
            f"Model {model_name} secret {secret_name} threshold {threshold} key {key}")
        print(">>Embedding...")
        watermarked_data, result = stego.embed(
            original_data, secret_data, threshold=threshold, secret_key=key)
        print(">>Extracting...")
        extracted_original_data, extracted_secret_data = stego.extract(
            watermarked_data, threshold=threshold, secret_key=key)

        print(">>", original_data[0:20], secret_data[0:20])
        print(">>", extracted_original_data[0:20],
              extracted_secret_data[0:20])
        print(">>", "Len -> Ori:", len(original_data), ", Ex-ori:", len(extracted_original_data),
              ", Sec:", len(secret_data), ", Ex-sec:", len(extracted_secret_data),
              f", Rate ({key}): {len(extracted_secret_data)*100/(key.count('1')*1200)}%")
        print("******\n******")

Secret untuk secret_0.39_bps
Model LASSO secret secret_0.39_bps threshold 4 key 001
>>Embedding...


LI: 111000000110 3590 12
[-145 -145 -145 -145 -145 -145 -145 -148 -116 -135]
Total yang lebih dari threshold (T = 4): 718
Unhidden secret: 997
NCC: 0.9999460987237585
PRD: 0.4902523544171775
SNR: 46.19160623946841
Time: 0.4917151927947998
>>Extracting...
LI: 111000000110 3590
[-145 -145 -145 -145 -145 -145 -145 -145 -120 -135]
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> Len -> Ori: 3600 , Ex-ori: 3600 , Sec: 1404 , Ex-sec: 407 , Rate (001): 33.916666666666664%
******
******
Model LASSO secret secret_0.39_bps threshold 4 key 010
>>Embedding...
LI: 111000001000 3592 12
[-145 -145 -145 -145 -145 -147 -142 -145 -116 -138]
Total yang lebih dari threshold (T = 4): 673
Unhidden secret: 974
NCC: 0.9999437022350083
PRD: 0.5038065479589854
SNR: 45.954723845770616
Time: 0.484208106994628

In [18]:
# Example
model_name = 'LASSO'
with open(models.get(model_name), 'rb') as model_file:
    model = pickle.load(model_file)

stego = MLPEEStego(model)
for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")
    for threshold in range(4, 10, 1):
        for key in ['001', '010', '011', '100', '101', '110']:
            print(
                f"Model {model_name} secret {secret_name} threshold {threshold} key {key}")
            print(">>Embedding...")
            watermarked_data, result = stego.embed(
                original_data, secret_data, threshold=threshold, secret_key=key)
            print(">>Extracting...")
            extracted_original_data, extracted_secret_data = stego.extract(
                watermarked_data, threshold=threshold, secret_key=key)

            print(">>", original_data[0:20], secret_data[0:20])
            print(">>", extracted_original_data[0:20],
                extracted_secret_data[0:20])
            print(">>", "Len -> Ori:", len(original_data), ", Ex-ori:", len(extracted_original_data),
                ", Sec:", len(
                    secret_data), ", Ex-sec:", len(extracted_secret_data),
                f", Rate ({key}): {len(extracted_secret_data)*100/(key.count('1')*1200)}%")
            print("******\n******")

Secret untuk secret_0.39_bps
Model LASSO secret secret_0.39_bps threshold 4 key 001
>>Embedding...
LI: 111000000110 3590 12
[-145 -145 -145 -145 -145 -145 -145 -148 -116 -135]
Total yang lebih dari threshold (T = 4): 718
Unhidden secret: 997
NCC: 0.9999460987237585
PRD: 0.4902523544171775
SNR: 46.19160623946841
Time: 0.4235808849334717
>>Extracting...
LI: 111000000110 3590
[-145 -145 -145 -145 -145 -145 -145 -145 -120 -135]
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11001001011110000011
>> Len -> Ori: 3600 , Ex-ori: 3600 , Sec: 1404 , Ex-sec: 407 , Rate (001): 33.916666666666664%
******
******
Model LASSO secret secret_0.39_bps threshold 4 key 010
>>Embedding...
LI: 111000001000 3592 12
[-145 -145 -145 -145 -145 -147 -142 -145 -116 -138]
Total yang lebih dari threshold (T = 4): 673
Unhidden secret: 